# Evaluation -> RAG Triad + Answer Correctness
## For -> LlaMa + Chroma (Embedding model used: BAAI/bge-large-en-v1.5)
### Shruti

In [1]:
!pip install pandas numpy transformers spacy

In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00


In [1]:
import spacy
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import string

# Load spaCy model for NLP tasks
nlp = spacy.load("en_core_web_sm")

# Load model and tokenizer
model_name = 'BAAI/bge-large-en-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/298A/m1_faiss_llama_output.csv')
df

,question,ground_truth,rag_answer,context,possible_questions
0,What are the keywords mentioned in the paper t...,The keywords written in the paper are: Earthqu...,The keywords mentioned in the paper are:\n\n1...,['Weights of the Approximating Artificial Neur...,['What are the key keywords mentioned in the p...
1,What is the purpose of using Artificial Neural...,Artificial Neural Networks (ANNs) are used to ...,The purpose of using Artificial Neural Networ...,"['method, that has been used here, is to creat...",['What is the main purpose of using Artificial...
2,How does the frequency of a building's natural...,A building's response to an earthquake is dyna...,The frequency of a building's natural frequen...,"['response, the building is subjected to a vib...","[""How does a building's natural frequency affe..."
3,What kind of neural network model is most freq...,The most frequently applied neural network mod...,"According to the text, the most frequently ap...",['storey structure. Among the different types ...,['What type of neural network model is most co...
4,How was the training of the ANN model conducte...,The ANN model was trained using real earthquak...,The training of the ANN model involved using ...,['Time period and the corresponding maximum re...,['What data was used to train the ANN model fo...
5,What is the significance of the study's findin...,The study's findings demonstrate the ability o...,The study's findings have significant implica...,['are accurate for practical purposes. When th...,"[""What are the key implications of the study's..."


In [4]:
# Function to calculate the embedding of a given text
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    # Reshape the tensor to remove the batch dimension and detach it from the computation graph
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().squeeze()

# Function to calculate cosine similarity between two embeddings
def calculate_similarity(embedding1, embedding2):
    # Ensure the embeddings are 2D
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

# Ccalculation for the advanced NLP tasks using spaCy
def calculate_context_relevancy(row):
    doc = nlp(row['context'])
    question_doc = nlp(row['question'])
    question_keywords = {token.lemma_.lower() for token in question_doc if not token.is_stop}
    relevant_sentences = sum(any(token.lemma_.lower() in question_keywords for token in sent) for sent in doc.sents)
    return relevant_sentences / len(list(doc.sents))

# Function to calculate answer relevancy
def calculate_answer_relevancy(row):
    original_question_embedding = get_embedding(row['question'])

    # 'possible_questions' is a list of artificially generated questions from the answer (reverse engineered)
    artificial_questions_embeddings = [get_embedding(q) for q in row['possible_questions']]

    relevancies = [
        calculate_similarity(original_question_embedding, q_emb)
        for q_emb in artificial_questions_embeddings
    ]

    # Calculate the mean of these relevancies to get the answer relevancy score
    answer_relevancy = np.mean(relevancies)
    return answer_relevancy

In [6]:
# Now apply these functions to each row in your dataframe
df['Context Relevance'] = df.apply(calculate_context_relevancy, axis=1)
#df['Answer Relevance'] = df.apply(calculate_answer_relevancy, axis=1)
df['Groundedness'] = df.apply(lambda row: calculate_similarity(get_embedding(row['rag_answer']), get_embedding(row['context'])), axis=1)
df['Answer Correctness'] = df.apply(lambda row: calculate_similarity(get_embedding(row['rag_answer']), get_embedding(row['ground_truth'])), axis=1)

# Display the updated DataFrame with the computed metrics
#df[['Context Relevance', 'Answer Relevance', 'Groundedness', 'Answer Correctness']]
df[['Context Relevance',  'Groundedness', 'Answer Correctness']]

,Context Relevance,Groundedness,Answer Correctness
0,0.538462,0.890928,0.887435
1,0.764706,0.898916,0.871439
2,0.736842,0.847566,0.936096
3,0.647059,0.898290,0.989992
4,0.823529,0.910430,0.894722
5,0.533333,0.915555,0.906734


In [11]:
# Now apply these functions to each row in your dataframe
df['Context Relevance'] = df.apply(calculate_context_relevancy, axis=1)
df['Answer Relevance'] = df.apply(calculate_answer_relevancy, axis=1)
df['Groundedness'] = df.apply(lambda row: calculate_similarity(get_embedding(row['rag_answer']), get_embedding(row['context'])), axis=1)
df['Answer Correctness'] = df.apply(lambda row: calculate_similarity(get_embedding(row['rag_answer']), get_embedding(row['ground_truth'])), axis=1)

# Display the updated DataFrame with the computed metrics
df[['Context Relevance', 'Answer Relevance', 'Groundedness', 'Answer Correctness']]

,Context Relevance,Answer Relevance,Groundedness,Answer Correctness
0,0.538462,0.532466,0.890928,0.887435
1,0.764706,0.490528,0.898916,0.871439
2,0.736842,0.488081,0.847566,0.936096
3,0.647059,0.500365,0.898290,0.989992
4,0.823529,0.507975,0.910430,0.894722
5,0.533333,0.514315,0.915555,0.906734


In [7]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/298A/m3_hybrid_rag_llama_output.csv')
df

,question,ground_truth,rag_answer,context,possible_questions
0,What are the keywords mentioned in the paper t...,The keywords written in the paper are: Earthqu...,The keywords mentioned in the paper are:\n\n1....,['Response Prediction of Structural System Sub...,['What are the key keywords mentioned in the p...
1,What is the purpose of using Artificial Neural...,Artificial Neural Networks (ANNs) are used to ...,"According to the text, the purpose of using Ar...",['intensities and it was found that the predic...,['What is the main purpose of using Artificial...
2,How does the frequency of a building's natural...,A building's response to an earthquake is dyna...,"When an earthquake occurs, the ground beneath ...","['buildings have a ""natural frequency"" associa...","[""How does a building's natural frequency affe..."
3,What kind of neural network model is most freq...,The most frequently applied neural network mod...,"According to the text, the most frequently app...",['Neural Networks for seismic damage predictio...,['What type of neural network model is most co...
4,How was the training of the ANN model conducte...,The ANN model was trained using real earthquak...,The training of the ANN model for predicting r...,['intensities and it was found that the predic...,['What data was used to train the ANN model fo...
5,What is the significance of the study's findin...,The study's findings demonstrate the ability o...,The study's findings on predicting the safenes...,['properties (mass and stiffness) of the struc...,"[""What are the key implications of the study's..."


In [8]:
# Now apply these functions to each row in your dataframe
df['Context Relevance'] = df.apply(calculate_context_relevancy, axis=1)
#df['Answer Relevance'] = df.apply(calculate_answer_relevancy, axis=1)
df['Groundedness'] = df.apply(lambda row: calculate_similarity(get_embedding(row['rag_answer']), get_embedding(row['context'])), axis=1)
df['Answer Correctness'] = df.apply(lambda row: calculate_similarity(get_embedding(row['rag_answer']), get_embedding(row['ground_truth'])), axis=1)

# Display the updated DataFrame with the computed metrics
#df[['Context Relevance', 'Answer Relevance', 'Groundedness', 'Answer Correctness']]
df[['Context Relevance',  'Groundedness', 'Answer Correctness']]

,Context Relevance,Groundedness,Answer Correctness
0,0.634921,0.876691,0.945441
1,0.615385,0.916004,0.918985
2,0.533333,0.843095,0.913964
3,0.597015,0.837430,0.990919
4,0.727273,0.921621,0.935274
5,0.240000,0.905882,0.920370


In [9]:
# Now apply these functions to each row in your dataframe
df['Context Relevance'] = df.apply(calculate_context_relevancy, axis=1)
df['Answer Relevance'] = df.apply(calculate_answer_relevancy, axis=1)
df['Groundedness'] = df.apply(lambda row: calculate_similarity(get_embedding(row['rag_answer']), get_embedding(row['context'])), axis=1)
df['Answer Correctness'] = df.apply(lambda row: calculate_similarity(get_embedding(row['rag_answer']), get_embedding(row['ground_truth'])), axis=1)

# Display the updated DataFrame with the computed metrics
df[['Context Relevance', 'Answer Relevance', 'Groundedness', 'Answer Correctness']]
#df[['Context Relevance',  'Groundedness', 'Answer Correctness']]

,Context Relevance,Answer Relevance,Groundedness,Answer Correctness
0,0.634921,0.532466,0.876691,0.945441
1,0.615385,0.490528,0.916004,0.918985
2,0.533333,0.488081,0.843095,0.913964
3,0.597015,0.500365,0.837430,0.990919
4,0.727273,0.507975,0.921621,0.935274
5,0.240000,0.514315,0.905882,0.920370
